In [20]:
import gymnasium as gym
import torch
from torch import nn as nn
from tqdm import tqdm

In [21]:
class ActorModel(nn.Module):
    def __init__(self, observation_size, hidden_size, out_size) -> None:
        super().__init__()
        self.activation = nn.LeakyReLU(0.2)
        self.lin1 = nn.Linear(observation_size, hidden_size)
        self.out = nn.Linear(hidden_size, out_size)
    
    def forward(self, state):
        out = self.lin1(state)
        out = self.activation(out)
        out = self.out(out)
        return nn.functional.softmax(out, dim=-1)


In [22]:
class Actor:
    def __init__(self, gamma, lr, device, observation_size, hidden_size, out_size) -> None:
        self.policy = ActorModel(observation_size, hidden_size, out_size)
        self.policy = self.policy.to(device)
        self.gamma = gamma
        self.optim = torch.optim.Adam(self.policy.parameters(), lr)

    def get_action(self, action):
        probs = self.policy(action)
        probs = probs.squeeze()
        distribution = torch.distributions.Categorical(probs)
        action = distribution.sample()
        log_prob = distribution.log_prob(action)
        return action, log_prob 
    
    def update(self, reward, next_state_value, state_value, log_prob):
        self.optim.zero_grad()
        loss = -(reward + self.gamma * next_state_value - state_value) * log_prob
        loss = torch.sum(loss)
        loss.backward()
        self.optim.step()

In [23]:
class CriticModel(nn.Module):
    def __init__(self, observation_size, hidden_size) -> None:
        super().__init__()
        self.activation = nn.ReLU()
        self.lin1 = nn.Linear(observation_size, hidden_size)
        self.lin3 = nn.Linear(hidden_size, 1)
    
    def forward(self, state):
        out = self.lin1(state)
        out = self.activation(out)
        return self.lin3(out)


In [24]:
class Critic:
    def __init__(self, gamma, lr, device, observation_size, hidden_size) -> None:
        self.value_function = CriticModel(observation_size, hidden_size)
        self.value_function = self.value_function.to(device)
        self.gamma = gamma
        self.optim = torch.optim.Adam(self.value_function.parameters(), lr)
    
    def get_state_value(self, state):
        return self.value_function(state)
    
    def update(self, reward, state_value, next_state_value):
        self.optim.zero_grad()
        loss = (reward + self.gamma * next_state_value - state_value).pow(2)
        loss.backward()
        self.optim.step()

In [25]:
class Agent:
    def __init__(self, env_name, gamma, actor_lr, critic_lr, device, hidden_size_actor, hidden_size_critic) -> None:
        self.device = device
        env = gym.make(env_name, max_episode_steps=500)
        self.test_env = gym.make(env_name, render_mode="human", max_episode_steps=500)
        self.env_wrapper = gym.wrappers.AutoResetWrapper(env)
        self.actor = Actor(gamma, actor_lr, device, observation_size=env.observation_space.shape[0], hidden_size=hidden_size_actor, out_size=env.action_space.n)
        self.critic = Critic(gamma, critic_lr, device, observation_size=env.observation_space.shape[0], hidden_size= hidden_size_critic)
    
    def learn(self, epochs):
        observation, info = self.env_wrapper.reset()
        observation = torch.from_numpy(observation).unsqueeze(0).to(self.device)

        rewards = []
        for epoch in tqdm(range(epochs)):
            ep_rewards = []

            done = False
            while not done:
                action, log_prob = self.actor.get_action(observation)
                action, log_prob = action.squeeze(), log_prob.squeeze()
                first_state_value = self.critic.get_state_value(observation)
                first_state_value = first_state_value.squeeze()

                observation, reward, terminated, truncated, info = self.env_wrapper.step(action.cpu().numpy())
                observation = torch.from_numpy(observation).unsqueeze(0).to(self.device)

                done = terminated or truncated
                if not done:
                    next_state_value = self.critic.get_state_value(observation)
                    next_state_value = next_state_value.squeeze()
                else:
                    next_state_value = torch.tensor(0)

                self.critic.update(reward=reward, state_value=first_state_value, next_state_value=next_state_value)
                self.actor.update(reward=reward, state_value=first_state_value.detach(), next_state_value=next_state_value.detach(), log_prob=log_prob)
                ep_rewards.append(reward)
                        
            rewards.append(sum(ep_rewards))
            
            if epoch % 100 == 0:
                for _ in range(10):
                    done = False
                    test_observation, info = self.test_env.reset()
                    test_observation = torch.from_numpy(test_observation).unsqueeze(0).to(self.device)
                    while not done:
                        with torch.no_grad():
                            action, _ = self.actor.get_action(test_observation)
                            action = action.squeeze()
                        test_observation, _, terminated, truncated, _ = self.test_env.step(action.cpu().numpy())
                        test_observation = torch.from_numpy(test_observation).unsqueeze(0).to(self.device)
                        done = terminated or truncated
                print(f"Episode {epoch}\t Mean reward = {sum(rewards)/len(rewards)}")
                rewards = []
        self.test_env.close()


In [26]:
agent = Agent(
    env_name="LunarLander-v2",
    gamma=0.99,
    actor_lr=0.0003,
    critic_lr=0.0003,
    device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"),
    hidden_size_actor=128,
    hidden_size_critic=128
)

In [27]:
agent.learn(5000)

  0%|          | 1/5000 [00:21<29:47:11, 21.45s/it]

Episode 0	 Mean reward = -44.37886048115726


  2%|▏         | 101/5000 [02:08<24:28:24, 17.98s/it]

Episode 100	 Mean reward = -188.90449342421743


  4%|▍         | 201/5000 [05:42<36:32:04, 27.41s/it]

...

 96%|█████████▌| 4801/5000 [3:25:11<1:46:01, 31.97s/it]

Episode 4800	 Mean reward = -43.70892358218991


 98%|█████████▊| 4901/5000 [3:29:20<47:21, 28.70s/it]  

Episode 4900	 Mean reward = -36.13497521507593


100%|██████████| 5000/5000 [3:31:53<00:00,  2.54s/it]
